In [1]:
import os
import csv
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timezone
import time
import alpaca_trade_api as tradeapi

import plotly.express as px

import sqlite3
from sqlite3 import Error


In [2]:
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
print(type(alpaca_api_key))

<class 'str'>


In [3]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [4]:
#Get tickers
tickers=[]
with open("sp10.csv",'r') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    header = next(csvreader)
    print(header)
    for row in csvreader:
        tickers.append(row[2])

print(tickers)


['ï»¿Number', 'Company', 'Symbol', 'Weight']
['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'TSLA', 'BRK.B', 'NVDA', 'JPM', 'JNJ']


In [5]:
end = datetime(2021,6,25,23,0,0,0,timezone.utc).isoformat()
start = datetime(2021,6,25,7,0,0,0,timezone.utc).isoformat()

In [12]:
db_file = "sp10_not_normalized.db"
try:
    conn = sqlite3.connect(db_file)
except Error as e:
    print(e)
df = pd.read_sql_query("SELECT * FROM historical", conn)
norm_df = df.pivot(index="timestamp", columns="symbol", values="close")
print(norm_df.head(10))
conn.close()

symbol                         AAPL      AMZN    BRK.B        FB    GOOGL  \
timestamp                                                                   
2017-01-03T09:00:00+00:00  116.0500  757.7600  164.500  116.3000  800.100   
2017-01-03T10:00:00+00:00  116.0200  757.8400      NaN  116.1600  800.000   
2017-01-03T11:00:00+00:00  116.2800  759.0000  164.900  116.4500  801.000   
2017-01-03T12:00:00+00:00  116.0200  759.3000  164.530  116.1500  800.000   
2017-01-03T13:00:00+00:00  116.2000  757.4000  164.150  116.1000  800.000   
2017-01-03T14:00:00+00:00  115.6700  753.8900  163.335  116.8702  806.160   
2017-01-03T15:00:00+00:00  115.4500  752.8999  163.140  117.2730  806.000   
2017-01-03T16:00:00+00:00  115.3757  750.5000  162.810  116.7790  808.470   
2017-01-03T17:00:00+00:00  115.4650  749.4500  162.800  116.8600  808.590   
2017-01-03T18:00:00+00:00  115.2350  748.7172  162.740  116.4869  805.195   

symbol                        JNJ      JPM   MSFT     NVDA     TSLA  
times

In [6]:
db_file = "sp10_not_normalized.db"
conn = sqlite3.connect(db_file)
sql_insert = "INSERT INTO historical(symbol, timestamp, close) VALUES(?,?,?)"
cur = conn.cursor()

# Grab data from Jan 1 2017 to June 25, 2021 by month and by ticker
# Loop will break after June 2021 and commit to database
# There is a time.sleep(5) after each API call to stay below API limits

for year in range(2017,2021+1):
    for month in range(1,12+1):
        if year == 2021 and month > 6:
            break
        temp_tz = pd.Timestamp(year=year, month=month, day=1)
        start = pd.Timestamp(f"{year}-{month}-{1}", tz="America/New_York").isoformat()
        if year == 2021 and month == 6:
            end = pd.Timestamp(f"{year}-{month}-{25}", tz="America/New_York").isoformat()
        else:
            end = pd.Timestamp(f"{year}-{month}-{temp_tz.daysinmonth}", tz="America/New_York").isoformat()
        print(start, end)

        for ticker in tickers:
            df = alpaca.get_bars(ticker, tradeapi.rest.TimeFrame.Hour, start=start, end=end, limit=1000).df
            time.sleep(5)
            for i in range(0,len(df)):
                cur.execute(sql_insert, (ticker, df.index[i].isoformat() , df.iloc[i]["close"]))

conn.commit()
conn.close()


2017-01-01T00:00:00-05:00 2017-01-31T00:00:00-05:00
2017-02-01T00:00:00-05:00 2017-02-28T00:00:00-05:00
2017-03-01T00:00:00-05:00 2017-03-31T00:00:00-04:00
2017-04-01T00:00:00-04:00 2017-04-30T00:00:00-04:00
2017-05-01T00:00:00-04:00 2017-05-31T00:00:00-04:00
2017-06-01T00:00:00-04:00 2017-06-30T00:00:00-04:00
2017-07-01T00:00:00-04:00 2017-07-31T00:00:00-04:00
2017-08-01T00:00:00-04:00 2017-08-31T00:00:00-04:00
2017-09-01T00:00:00-04:00 2017-09-30T00:00:00-04:00
2017-10-01T00:00:00-04:00 2017-10-31T00:00:00-04:00
2017-11-01T00:00:00-04:00 2017-11-30T00:00:00-05:00
2017-12-01T00:00:00-05:00 2017-12-31T00:00:00-05:00
2018-01-01T00:00:00-05:00 2018-01-31T00:00:00-05:00
2018-02-01T00:00:00-05:00 2018-02-28T00:00:00-05:00
2018-03-01T00:00:00-05:00 2018-03-31T00:00:00-04:00
2018-04-01T00:00:00-04:00 2018-04-30T00:00:00-04:00
2018-05-01T00:00:00-04:00 2018-05-31T00:00:00-04:00
2018-06-01T00:00:00-04:00 2018-06-30T00:00:00-04:00
2018-07-01T00:00:00-04:00 2018-07-31T00:00:00-04:00
2018-08-01T0

In [20]:
# db_file = "sp10_not_normalized.db"
# conn = sqlite3.connect(db_file)
# select = "SELECT DISTINCT timestamp FROM historical ORDER BY timestamp ASC"
# cur = conn.cursor()
# cur.execute(select)

# dates = []
# for timestamp in cur:
#     dates.append(timestamp[0])



# conn.commit()
# conn.close()
# print(dates[0:10])

['2017-01-03T09:00:00+00:00', '2017-01-03T10:00:00+00:00', '2017-01-03T11:00:00+00:00', '2017-01-03T12:00:00+00:00', '2017-01-03T13:00:00+00:00', '2017-01-03T14:00:00+00:00', '2017-01-03T15:00:00+00:00', '2017-01-03T16:00:00+00:00', '2017-01-03T17:00:00+00:00', '2017-01-03T18:00:00+00:00']


In [32]:
(df_list[0])

128